# Lab 7 — Zarzadzanie zrodlami danych strumieniowych i segmentacja klientow

## 🔄 Wprowadzenie

W tym laboratorium zapoznasz sie z roznymi metodami zasilania danych strumieniowych w Apache Spark oraz zastosowaniem prostych transformacji, filtrowania i segmentacji klientow w czasie rzeczywistym.

---

## 💡 Pomocnicza funkcja do wyswietlania naszych danych strumieniowych


In [1]:
batch_counter = {"count": 0}

def process_batch(df, batch_id):
    batch_counter["count"] += 1
    print(f"Batch ID: {batch_id}")
    df.show(truncate=False)

---

## 🔹 rate jako źródło kontrolowanego strumienia

### ✅ Zadanie 1

1. Przygotuj strumien danych z `format('rate')`, ustaw `rowsPerSecond` na 5.
2. Utworz kolumne `user_id`: `expr("concat('u', cast(rand()*100 as int))")`
3. Dodaj kolumne `event_type`: `expr("case when rand() > 0.7 then 'purchase' else 'view' end")`

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.builder.appName("StreamingDemo").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# Źródło danych typu rate
rate_df = (spark.readStream
           .format("rate")
           .option("rowsPerSecond", 5)
           .load())

# Dodanie kolumn user_id i event_type
events = (rate_df
          .withColumn("user_id", expr("concat('u', cast(rand()*100 as int))"))
          .withColumn("event_type", expr("case when rand() > 0.7 then 'purchase' else 'view' end")))

# Zapis strumienia z wykorzystaniem process_batch
query = (events.writeStream
         .format("console")
         .foreachBatch(process_batch)
         .start())

query.awaitTermination()


25/05/19 19:01:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6ee3fa96-6bd0-4e36-8317-75aee69f7823. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/19 19:01:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.python/current/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-

KeyboardInterrupt: 

---

## 🔹 Filtrowanie danych bez agregacji (append mode)

Zobacz jak działa poniższy kod i na jego podstawie:

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("AppendExample").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# Źródło rate - generuje timestamp + value
rate_df = (spark.readStream
           .format("rate")
           .option("rowsPerSecond", 5)
           .load())

# Filtracja bez potrzeby agregacji (bezstanowe przetwarzanie)
filtered = rate_df.filter(col("value") % 2 == 0) \
                  .withColumn("info", expr("concat('even:', value)"))

# outputMode = append → pokazuje tylko nowe wiersze, bez stanu
query = (filtered.writeStream 
    .outputMode("append") 
    .format("console") 
    .option("truncate", False) 
    .foreachBatch(process_batch)
    .start()
        )
```



1. Skorzystaj z danych z poprzedniego zadania.
2. Wyfiltruj tylko `purchase`.

In [6]:
# Filtrowanie tylko 'purchase'
purchases = events.filter(col("event_type") == "purchase")

query = (purchases.writeStream
         .format("console")
         .outputMode("append")
         .foreachBatch(process_batch)
         .start())

25/05/19 18:56:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c6300fe0-375d-47f9-9e1d-74c9b8536008. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/19 18:56:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


---

## 🔹 Źródło plikowe (JSON)

### ✅ Generator danych:

In [13]:
%%file generator.py
# generator.py
import json, os, random, time
from datetime import datetime, timedelta

output_dir = "data/stream"
os.makedirs(output_dir, exist_ok=True)

event_types = ["view", "cart", "purchase"]
categories = ["electronics", "books", "fashion", "home", "sports"]

def generate_event():
    return {
        "user_id": f"u{random.randint(1, 50)}",
        "event_type": random.choices(event_types, weights=[0.6, 0.25, 0.15])[0],
        "timestamp": (datetime.utcnow() - timedelta(seconds=random.randint(0, 300))).isoformat(),
        "product_id": f"p{random.randint(100, 120)}",
        "category": random.choice(categories),
        "price": round(random.uniform(10, 1000), 2)
    }

# Simulate file-based streaming
while True:
    batch = [generate_event() for _ in range(50)]
    filename = f"{output_dir}/events_{int(time.time())}.json"
    with open(filename, "w") as f:
        for e in batch:
            f.write(json.dumps(e) + "\n")
    print(f"Wrote: {filename}")
    time.sleep(5)

Overwriting generator.py


### ✅ Schemat danych:

```json
{
  "user_id": "u123",
  "event_type": "purchase", // albo "view", "cart", "click"
  "timestamp": "2025-05-09T15:24:00Z",
  "product_id": "p456",
  "category": "electronics",
  "price": 299.99
}
```

1. Utwórz zmienną `schema`, która zrealizuje schamat danych naszej ramki. Wykorzystaj `StringType()`, `TimestampType()`,`DoubleType()`


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("RealTimeEcommerce").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# StringType(), TimestampType(), DoubleType()

from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType

schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("event_type", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("product_id", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True)
])

### ✅ Odczyt danych z katalogu:

In [15]:
stream = (spark.readStream
          .schema(schema)
          .json("data/stream"))

query = (stream.writeStream
         .format("console")
         .foreachBatch(process_batch)
         .start())

25/05/19 19:06:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-dc0ee2cb-1c4b-45c1-acf4-dd7db15bccf5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/19 19:06:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


---

## 🔹 Bezstanowe zliczanie zdarzen

1. Przygotuj zmienną agg1 zliczającą zdarzenia należące do danej grupy `event_type`.

In [16]:
from pyspark.sql.functions import col, count

# Agregacja liczby zdarzeń w grupie event_type
agg1 = (stream
        .groupBy("event_type")
        .agg(count("*").alias("event_count")))

# pamietaj, że agregacje wymagają opcji complete
query = (agg1
         .writeStream
         .outputMode("complete")
         .format("console")
         .foreachBatch(process_batch)
         .start()
        )

25/05/19 19:06:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8578a163-0f39-4152-bafd-926b7e0f3cbd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/19 19:06:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


---

## 🔹 Agregacja w oknach czasowych

`withWatermark("timestamp", "1 minute")`
   
💡 Do czego służy: Informuje Sparka, że dane przychodzą z opóźnieniem i należy je przetwarzać tylko do określonego limitu wstecz (tutaj: 1 minuta).

🚨 Dlaczego ważne: Bez watermarku Spark trzymałby w pamięci wszystkie dane, by móc je jeszcze pogrupować. Watermark pozwala zwolnić pamięć.

1. Pogrupuj typy zdarzen w thumbling window, w oknie co 5 minut
2. dodaj watermark z ustawieniem na 1 minutę. 

In [17]:
windowed = (stream
              .withWatermark("timestamp", "1 minute")
              .groupBy(
                  window(col("timestamp"), "5 minutes"),
                  col("event_type")
              )
              .agg(count("*").alias("event_count"))
             )

query = (
    windowed.writeStream
    .outputMode("append")
    .foreachBatch(process_batch)
    .format("console")
    .start()
)

25/05/19 19:06:58 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3e9a6abd-6cc7-409e-bfe5-48340986b087. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/19 19:06:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


3. Zmień thumbling window na sliding window z szerokością okna 5 minut i startem nowego okna co 1 minutę. 

In [18]:
windowed = (stream
               .withWatermark("timestamp", "1 minute")
               .groupBy(
                   window(col("timestamp"), "5 minutes", "1 minute"),
                   col("event_type")
               )
               .agg(count("*").alias("event_count"))
              )

query = (
    windowed.writeStream
    .outputMode("append")
    .foreachBatch(process_batch)
    .format("console")
    .start()
)

25/05/19 19:07:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-68ee8acc-6cb2-4215-bae4-6f22eebbd787. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/19 19:07:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.



---

## 🔹 Segmentacja klientow

🧩 Logika segmentacji:

1. jeśli był purchase → "Buyer"
2. jeśli był cart, ale nie purchase → "Cart abandoner"
3. jeśli tylko view → "Lurker"


`groupBy(window(...), "user_id")`

💡 Do czego służy: Grupujemy dane per użytkownik w konkretnym przedziale czasu (oknie 5-minutowym).

⏱️ window("timestamp", "5 minutes"): Funkcja okna czasowego – każda grupa będzie dotyczyć jednego użytkownika w konkretnym 5-minutowym interwale.

`agg(collect_set("event_type"))`

💡 Do czego służy: Zbiera wszystkie typy zdarzeń (view, cart, purchase) danego użytkownika w danym oknie.

🧠 Dlaczego `collect_set` a nie `collect_list`?: collect_set usuwa duplikaty — interesuje nas tylko czy coś się zdarzyło, a nie ile razy.

5. withColumn(... expr(...))

💡 Do czego służy: Na podstawie zbioru zdarzeń określamy segment użytkownika.

🔎 `array_contains`: Funkcja sprawdzająca, czy dany typ zdarzenia znajduje się w tablicy.




🧠 Co warto wiedzieć:

- Segmentacja to klasyczne zastosowanie agregacji i transformacji strumienia.
- Łączenie window + watermark jest kluczowe do kontroli stanu.
- collect_set umożliwia prostą analizę zachowań, bez potrzeby przechowywania surowych danych.
- expr() daje elastyczność, by używać składni SQL wewnątrz kodu DataFrame.

In [19]:
from pyspark.sql.functions import window, col, collect_set, expr

# Agregacja po 5-minutowym oknie i user_id,
agg_segmentation = (stream
                    .withWatermark("timestamp", "1 minute")
                    .groupBy(
                        window(col("timestamp"), "5 minutes"),
                        col("user_id")
                    )
                    .agg(collect_set("event_type").alias("events"))
                   )

# Definiujemy segment na podstawie występowania event_type w tablicy
segmented = agg_segmentation.withColumn(
    "segment",
    expr("""
        CASE
            WHEN array_contains(events, 'purchase') THEN 'Buyer'
            WHEN array_contains(events, 'cart') THEN 'Cart abandoner'
            ELSE 'Lurker'
        END
    """)
)


---
## ⚙️ Porownanie trybow `outputMode`

| outputMode | Opis                                        | Kiedy uzywac                           | Wymagania                 |
| ---------- | ------------------------------------------- | -------------------------------------- | ------------------------- |
| `append`   | Wypisywane sa **tylko nowe wiersze**        | Filtrowanie, wzbogacanie bez agregacji | Nie dziala z `groupBy`    |
| `update`   | Wypisywane sa tylko **zmienione wiersze**   | Agregacje z watermarkami               | Wymaga zarzadzania stanem |
| `complete` | Wypisywane jest **calosciowe podsumowanie** | Podsumowania okien, snapshoty          | Moze byc kosztowne        |

---